# Basics of network design

In [3]:
# This is only valid when the package is not installed
import sys
sys.path.append('../../') # two folders up
import DeepINN as dp
import torch

## Geometry

In [4]:
# A simple 1D geometry
X = dp.spaces.R1('x')
Line = dp.domains.Interval(X, 0, 1)

In [5]:
left_bc = dp.constraint.DirichletBC(geom = Line,
                                      function = lambda X: torch.tensor([0.0]),
                                      sampling_strategy = "grid",
                                      no_points = 1, # you can use more points. there are conditions to deal with stupid conditions. 
                                      filter_fn = lambda x: x[:] == 0.0)
right_bc = dp.constraint.DirichletBC(geom = Line,
                                      function = lambda X: torch.tensor([1.0]),
                                      sampling_strategy = "grid",
                                      no_points = 1, # you can use more points. there are conditions to deal with stupid conditions. 
                                      filter_fn = lambda x: x[:] == 1.0)
interior_points = dp.constraint.PDE(geom = Line,
                                    sampling_strategy= "grid",
                                    no_points = 20)

In [6]:
# dp.utils.scatter(X, interior_points.sampler_object(), dpi = 50)
# dp.utils.scatter(X, left_bc.sampler_object(), dpi = 50)
# dp.utils.scatter(X, right_bc.sampler_object(), dpi = 50)

## 1D Laplace equation

In [7]:
def laplace(X,y):
    """
    1D Laplace equation.
    u__x = 0 
    """
    dy_x = dp.constraint.Jacobian(X, y)(i = 0, j = 0)
    dy_xx = dp.constraint.Jacobian(X, dy_x)(i = 0, j = 0)

    return dy_xx


## Dataset

In [8]:
dataset = dp.domain.Generic(laplace,
                            interior_points,
                            [left_bc, right_bc])

In [9]:
collocation_point_sample, collocation_point_labels = dataset.sample_collocation_labels()
collocation_point_sample.size(), collocation_point_labels.size()

(torch.Size([20, 1]), torch.Size([20, 1]))

In [10]:
# Don't use this sample_boundary_labels() returns both inputs and labels
boundary_point_sample = dataset.sample_boundary_points()
boundary_point_sample

/home/hell/Desktop/repos/DeepINN/Tutorials/5. FCNN/../../DeepINN/geometry/samplers/grid_samplers.py:78: UserWarning: First iteration did not find any valid grid points, for
                             the given filter.
                             Will try again with n = 10 * self.n_points. Or
                             else use only random points!
  warnings.warn("""First iteration did not find any valid grid points, for


[tensor([[0.]]), tensor([[1.]])]

In [11]:
boundary_point_sample, boundary_point_labels = dataset.sample_boundary_labels()
len(boundary_point_sample), len(boundary_point_labels)

(2, 2)

In [12]:
boundary_point_labels

[tensor([[0.]]), tensor([[1.]])]

In [13]:
boundary_point_labels[0].size()

torch.Size([1, 1])

In [14]:
boundary_point_sample[0].requires_grad = True

# y is the output neuron of a hypothetical neural network
# the output and the input should be connected to the graph to enable chain rule.
y = boundary_point_sample[0] **2# + boundary_point_labels[0]

In [15]:
laplace(boundary_point_sample[0],y)

tensor([[2.]], grad_fn=<IndexBackward0>)